In [1]:
import time
import mlrose_hiive as mlrh
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

random_state = 923
np.random.seed(923)

# four peaks Problem

In [4]:
# Problem definition
length = 150
t_pct = 0.9
global eval_count
eval_count = 0

base_fit_func = mlrh.FourPeaks(t_pct=t_pct).evaluate

def four_peaks_fit(state):
    global eval_count
    eval_count += 1
    return base_fit_func(state)

# Initialize custom fitness function object
fitness = mlrh.CustomFitness(four_peaks_fit)

problem = mlrh.DiscreteOpt(length=length,
                        fitness_fn=fitness,
                        maximize=True)
experiment_name = "four_peaks"
output_directory = "four_peaks"

## MIMIC

In [5]:
mim = mlrh.MIMICRunner(problem=problem,
                       experiment_name=experiment_name,
                       output_directory=output_directory,
                       seed=random_state,
                       population_sizes=[50, 100, 200, 500, 1000, 2000],
                       keep_percent_list=[0.1, 0.25, 0.5, 0.7, 0.9],
                       iteration_list=[50],
                       use_fast_mimic=True)
mim_stats, mim_curve = mim.run()
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Keep Percent']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./four_peaks/four_peaks/mimic__four_peaks__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Population Size'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Keep Percent
51,35.500991,79.0,2000,0.10
53,32.058161,62.0,2000,0.25
41,21.996381,54.0,1000,0.10
43,19.244551,53.0,1000,0.25
33,10.448002,51.0,500,0.25
21,6.034269,31.0,200,0.10
31,12.678757,30.0,500,0.10
23,5.808192,24.0,200,0.25
55,56.565898,23.0,2000,0.50
35,14.253350,21.0,500,0.50


## Genetic Algorithm

In [6]:
ga = mlrh.GARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=20,
                   population_sizes=[50, 100, 200, 500, 1000, 2000],
                   mutation_rates=[0.1, 0.25, 0.5, 0.7, 0.9],
                   iteration_list=[50])
ga_stats, ga_curve = ga.run()
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Mutation Rate']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./four_peaks/four_peaks/ga__four_peaks__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Mutation Rate'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Mutation Rate
55,7.250871,57.0,2000,0.50
53,7.115957,55.0,2000,0.25
59,7.340177,55.0,2000,0.90
43,3.241266,54.0,1000,0.25
45,3.270219,54.0,1000,0.50
39,1.668460,52.0,500,0.90
37,1.566455,51.0,500,0.70
33,1.422474,49.0,500,0.25
47,3.338246,49.0,1000,0.70
49,3.363714,49.0,1000,0.90


## Simulated Annealing

In [7]:
sa = mlrh.SARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=200,
                   iteration_list=[1000],
                   temperature_list=[0.05, 0.1, 0.5, 1, 10, 30],
                   decay_list=[mlrh.GeomDecay, mlrh.ExpDecay, mlrh.ArithDecay])
sa_stats, sa_curve = sa.run()

show=10
columns = ['Time', 'Fitness', 'Temperature', 'schedule_type']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./four_peaks/four_peaks/sa__four_peaks__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Temperature'], ascending=(False,True)).head(show)

,Time,Fitness,Temperature,schedule_type
25,0.239293,13.0,10.00,geometric
1,0.190933,12.0,0.05,geometric
3,0.184274,12.0,0.05,exponential
5,0.180826,12.0,0.05,arithmetic
7,0.196562,12.0,0.10,geometric
9,0.201275,12.0,0.10,exponential
11,0.204015,12.0,0.10,arithmetic
13,0.209200,12.0,0.50,geometric
15,0.226526,12.0,0.50,exponential
17,0.219422,12.0,0.50,arithmetic


# Randomized Hill Climbing

In [8]:
rhc = mlrh.RHCRunner(problem=problem,
                    experiment_name=experiment_name,
                    output_directory=output_directory,
                    seed=random_state,
                    max_attempts=100,
                    iteration_list=[1000],
                    restart_list=[5,10])
rhc_stats, rhc_curve = rhc.run()
show=10
columns = ['Time', 'Fitness', 'Restarts', 'current_restart']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./four_peaks/four_peaks/rhc__four_peaks__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Restarts'], ascending=(False,True)).head(show)

,Time,Fitness,Restarts,current_restart
33,0.940655,10.0,10,10
32,0.853890,9.0,10,10
27,0.592180,8.0,10,7
29,0.683433,6.0,10,8
1,0.034092,5.0,5,0
13,0.028732,5.0,10,0
31,0.824835,5.0,10,9
2,0.054034,3.0,5,1
3,0.081797,3.0,5,1
10,0.282850,3.0,5,5
